In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
df_ords_prods_merge = (pd.read_pickle(r'C:\Users\celin\OneDrive\Desktop\Career Foundry\Instacart Basket Analysis 5.23\02 Data\Prepared data\df_combined_added_frequency_columns.pkl'))

In [3]:
path = r'C:\Users\celin\OneDrive\Desktop\Career Foundry\Instacart Basket Analysis 5.23'

In [4]:
# Step 2 - aggregate mean for entire dataframe:
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [5]:
df_dept = pd.read_csv(r'C:\Users\celin\OneDrive\Desktop\Career Foundry\Instacart Basket Analysis 5.23\02 Data\Prepared data\departments_wrangled.csv', index_col = False)

In [6]:
df_dept

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [ ]:
# Step 3 - Most of the mean values in the subset were values around 17 and 19, with an outlier value of 11. 
# Looking at the full dataset, the values are similar, most mean values are around 16/17 
# 10 and 21 are highest, 10 being 'bulk' and 21 is 'missing', which could explain the mean value at around 23 if 'missing' items are mispriced/mislabeled. 

In [14]:
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [15]:
# Step 4 - Create flag for existing customer (as outlined in practice task):
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [16]:
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [17]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [18]:
# Conduct check:
df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [19]:
# Step 5 - Spending habits of customers:
df_ords_prods_merge.groupby('prices')

In [20]:
df_ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,10.386336
New customer,13.294670
Regular customer,12.495717


In [ ]:
# It appears that the most loyal customers purchase less per orders but order more often. 

In [ ]:
# Step 6 - The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [21]:
df_ords_prods_merge['mean_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [22]:
df_ords_prods_merge.loc[df_ords_prods_merge['mean_price'] >= 10, 'spender_flag'] = 'High spender'

In [23]:
df_ords_prods_merge.loc[df_ords_prods_merge['mean_price'] <10, 'spender_flag'] = 'Low spender'

In [24]:
df_ords_prods_merge['spender_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spender_flag, dtype: int64

In [25]:
df_ords_prods_merge.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,mean_order,loyalty_flag,max_order,mean_price,spender_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders,5.813559,New customer,10,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Mid-range product,Regularly busy,Least busy days,Average orders,5.813559,New customer,10,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Mid-range product,Regularly busy,Least busy days,Most orders,5.813559,New customer,10,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Mid-range product,Least busy,Least busy days,Average orders,5.813559,New customer,10,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Mid-range product,Least busy,Least busy days,Most orders,5.813559,New customer,10,6.367797,Low spender
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders,5.813559,New customer,10,6.367797,Low spender
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,Mid-range product,Mid-range product,Regularly busy,Busiest days,Most orders,5.813559,New customer,10,6.367797,Low spender
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,Mid-range product,Mid-range product,Regularly busy,Busiest days,Most orders,5.813559,New customer,10,6.367797,Low spender
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,Mid-range product,Mid-range product,Regularly busy,Busiest days,Most orders,5.813559,New customer,10,6.367797,Low spender
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,Mid-range product,Mid-range product,Least busy,Least busy days,Average orders,5.813559,New customer,10,6.367797,Low spender


In [ ]:
# Step 7: In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [26]:
df_ords_prods_merge['frequent_customer'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [27]:
df_ords_prods_merge.loc[df_ords_prods_merge['frequent_customer'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [28]:
df_ords_prods_merge.loc[(df_ords_prods_merge['frequent_customer'] > 10) & (df_ords_prods_merge['frequent_customer'] <=20), 'frequency_flag'] = 'Regular customer'

In [29]:
df_ords_prods_merge.loc[df_ords_prods_merge['frequent_customer'] <=10, 'frequency_flag'] = 'Frequent customer'

In [30]:
df_ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [ ]:
# NaN value (5) appears to be first orders customers, median cannot be calculated and those customers would be labeled 'Non-frequent customers' as well. 

In [31]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_aggrevated.pkl'))